In [5]:
import pandas as pd
import glob
import gc

## Functions and Variables

In [1]:
def find_files(path, pattern):
    files = glob.glob(f'{path}/{pattern}')
    # print(files)
    # print(len(files))

    return files

In [2]:
# Creates a column called index in the dataframe
# this column will have the format:
# prefix + 0000001 (Ex. 310.000.001) to prefix + length of dataframe (Ex. 312.123.942)

def add_index(df, prefix):
    size = len(df)
    prefix = prefix * 10**len(str(size)) + 1

    df['index'] = range(prefix, prefix + size)
    df.set_index('index', inplace=True)

In [3]:
def safe_delete(df, list_columns):
    for item in list_columns:
        if item in df.columns:
            df = df.drop(item, axis=1)
    
    return df

In [6]:
files = find_files('D:\\_repositories\\Aggregation\\SINASC - Dtype Corrigido\\parquet', 
                   '*.parquet')
files

['D:\\_repositories\\Aggregation\\SINASC - Dtype Corrigido\\parquet\\DN2012.parquet',
 'D:\\_repositories\\Aggregation\\SINASC - Dtype Corrigido\\parquet\\DN2013.parquet',
 'D:\\_repositories\\Aggregation\\SINASC - Dtype Corrigido\\parquet\\DN2014.parquet',
 'D:\\_repositories\\Aggregation\\SINASC - Dtype Corrigido\\parquet\\DN2015.parquet',
 'D:\\_repositories\\Aggregation\\SINASC - Dtype Corrigido\\parquet\\DN2016.parquet',
 'D:\\_repositories\\Aggregation\\SINASC - Dtype Corrigido\\parquet\\DN2017.parquet',
 'D:\\_repositories\\Aggregation\\SINASC - Dtype Corrigido\\parquet\\DN2018.parquet',
 'D:\\_repositories\\Aggregation\\SINASC - Dtype Corrigido\\parquet\\DN2019.parquet',
 'D:\\_repositories\\Aggregation\\SINASC - Dtype Corrigido\\parquet\\DN2020.parquet',
 'D:\\_repositories\\Aggregation\\SINASC - Dtype Corrigido\\parquet\\DN2021.parquet',
 'D:\\_repositories\\Aggregation\\SINASC - Dtype Corrigido\\parquet\\DN2022.parquet']

---

## Processing

In [9]:
inicial_ano = 12 # ano em processamento
lista_incompleto = []

for item in files:
    df = pd.read_parquet(item)

    # Index
    add_index(df, int('2' + str(inicial_ano)))

    # Deleting past index columns
    df = safe_delete(df, ['Unnamed: 0', 'Unnamed: 0.1'])

    # Saving to parquet
    try: 
        df.to_parquet(f'D:/_repositories/Aggregation/SINASC - Index adicionado/DN20{inicial_ano}.parquet')
    except pa.ArrowInvalid:
        lista_incompleto.append(item)
    
    inicial_ano += 1

    # Memory safety
    del df
    gc.collect()